In [33]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df1 = pd.read_csv("data4_미포함.csv", index_col = 0)
df2 = pd.read_csv("data4_포함.csv", index_col = 0)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17773 entries, 0 to 17772
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   회사명                             17773 non-null  object 
 1   거래소코드                           17773 non-null  int64  
 2   회계년도                            17773 non-null  object 
 3   [제조]총자본순이익률                     17322 non-null  float64
 4   [제조]총자본영업이익률                    17322 non-null  float64
 5   [제조]자기자본순이익률                    17322 non-null  float64
 6   [제조]매출액총이익률                     17322 non-null  float64
 7   [제조]매출액영업이익률                    17322 non-null  float64
 8   [제조]금융비용부담률                     17322 non-null  float64
 9   [제조]자기자본배율                      17322 non-null  float64
 10  [제조]부채비율                        17322 non-null  float64
 11  [제조]유동비율                        17322 non-null  float64
 12  [제조]당좌비율                        

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19989 entries, 0 to 19988
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   회사명                             19989 non-null  object 
 1   거래소코드                           19989 non-null  int64  
 2   회계년도                            19989 non-null  object 
 3   [제조]총자본순이익률                     19489 non-null  float64
 4   [제조]총자본영업이익률                    19489 non-null  float64
 5   [제조]자기자본순이익률                    19489 non-null  float64
 6   [제조]매출액총이익률                     19489 non-null  float64
 7   [제조]매출액영업이익률                    19489 non-null  float64
 8   [제조]금융비용부담률                     19489 non-null  float64
 9   [제조]자기자본배율                      19489 non-null  float64
 10  [제조]부채비율                        19489 non-null  float64
 11  [제조]유동비율                        19489 non-null  float64
 12  [제조]당좌비율                        

In [6]:
df1['거래소코드'] = df1.거래소코드.map('{:06d}'.format)
df2['거래소코드'] = df2.거래소코드.map('{:06d}'.format)

In [17]:
# 폐지사를 찾기 위해 두 데이터를 거래소코드만을 가져와 비교함.
# df1을 거래소코드로 정렬
to_dataframe = np.array(pd.unique(df1['거래소코드']))
df1_code = pd.DataFrame(to_dataframe, columns = ['거래소코드'])
df1_code.sort_values(['거래소코드'], ascending = True, inplace = True)
df1_code.reset_index(inplace = True)
df1_code.set_index(['거래소코드'], inplace = True)
df1_code.rename(columns = {'index' : 'index of df1'}, inplace = True)

In [18]:
# df2를 거래소코드로 정렬
to_dataframe2 = np.array(pd.unique(df2['거래소코드']))
df2_code = pd.DataFrame(to_dataframe2, columns = ['거래소코드'])
df2_code.sort_values(['거래소코드'],ascending = True, inplace = True)
df2_code.reset_index(inplace = True)
df2_code.set_index(['거래소코드'], inplace = True)
df2_code.rename(columns = {'index' : 'index of df2'}, inplace = True)

In [19]:
df = pd.concat([df1_code, df2_code], axis = 1)
df

,index of df1,index of df2
거래소코드,,
000020,1212.0,1558
000040,1593.0,2008
000050,19.0,21
000070,292.0,371
000080,1657.0,2095
...,...,...
900060,NaN,1981
900150,NaN,1695
950010,NaN,2078


In [20]:
# df1에 nan 값만 추출하여 폐지사 선정
new_df = df[df['index of df1'].isna()]
new_df

,index of df1,index of df2
거래소코드,,
000360,NaN,1676
000420,NaN,238
000790,NaN,520
000800,NaN,1413
000830,NaN,1645
...,...,...
900060,NaN,1981
900150,NaN,1695
950010,NaN,2078


In [22]:
# 폐지사 출력
li = np.array(new_df.index)
dff = pd.DataFrame(li, columns = ['거래소코드'])
dff

,거래소코드
0,000360
1,000420
2,000790
3,000800
4,000830
...,...
445,900060
446,900150
447,950010
448,950030


In [23]:
# 폐지사 중 감사의견코드 기준(DS, DU)는 부도 확정이므로 제거
merge = pd.merge(dff, df2[['거래소코드', '회계년도', '감사의견코드']], how = 'inner', on = '거래소코드')
merge

,거래소코드,회계년도,감사의견코드
0,000360,2010/12,UQ
1,000360,2011/12,UQ
2,000360,2012/12,UQ
3,000360,2009/12,UQ
4,000360,2008/12,UQ
...,...,...,...
2166,950030,2011/03,UQ
2167,950070,2010/12,NaN
2168,950070,2009/12,NaN
2169,950070,2011/12,NaN


In [24]:
len(merge.거래소코드.unique())

450

In [35]:
# 각 거래소코드 중 가장 마지막년도의 감사의견코드를 확인하기 위한 코드
merge.sort_values(by = ['거래소코드', '회계년도'], inplace = True)
mer = merge.groupby(['거래소코드'])['회계년도', '감사의견코드'].last().reset_index()
mer

,거래소코드,회계년도,감사의견코드
0,000360,2014/12,UQ
1,000420,2013/12,DU
2,000790,2008/12,DU
3,000800,2014/12,DU
4,000830,2014/12,UQ
...,...,...,...
445,900060,2012/03,None
446,900150,2011/12,None
447,950010,2014/12,None
448,950030,2011/03,UQ


In [36]:
# 선정 부도 기준인 감사의견코드('DS, DU')
budo = mer[(mer['감사의견코드']=='DS') | (mer['감사의견코드']=='DU')]
len(budo.거래소코드.unique())

178

In [37]:
# 선정 부도 기준인 감사의견코드('DS, DU')에 해당하지 않는 데이터를 추출
budoYN = mer[(mer['감사의견코드']!='DS') & (mer['감사의견코드']!='DU')]
budoYN.감사의견코드.unique()

array(['UQ', None, 'QS', 'QU', 'AG', 'QG'], dtype=object)

In [63]:
# 수작업을 위한, 부도가 확정되지 않은 기업 데이터 추출
to_dataframe = np.array(pd.unique(budoYN['거래소코드']))
df = pd.DataFrame(to_dataframe, columns = ['거래소코드'])
df.sort_values(['거래소코드'], ascending = True, inplace = True)
df.reset_index(inplace = True)
df.set_index(['거래소코드'], inplace =True)
df

,index
거래소코드,
000360,0
000830,1
001300,2
001310,3
001370,4
...,...
900060,267
900150,268
950010,269


In [41]:
# DART 공시를 활용하여 직접 부도 여부를 조사함(ex:합병 등은 부도에서 제외)
li = np.array(df.index)
dff = pd.DataFrame(li, columns = ['거래소코드'])
dff

,거래소코드
0,000360
1,000830
2,001300
3,001310
4,001370
...,...
267,900060
268,900150
269,950010
270,950030


In [58]:
# 중간에 데이터의 변경으로 지난 수작업 자료에서 부도 라벨링 한 것 중 중복되는 것을 가져와 불필요한 작업을 피하기 위한 코딩
# 부도라벨링으로 부도확정된 기업을 가져옴
pezi = pd.read_excel('전체 폐지사(부도데이터).xlsx', header = None)
pezi.rename(columns = {0 : 'index', 1 : '거래소코드', 2 : '부도여부',
                      3 : '회계년도', 4 : '폐지사유'}, inplace = True)

pezi.drop(columns = {'index', '회계년도', '폐지사유'}, axis = 1, inplace = True)
pezi.거래소코드 = pezi.거래소코드.astype('int')
pezi.거래소코드 = pezi.거래소코드.map('{:06d}'.format)
pezi.set_index(['거래소코드'], inplace = True)
pezi.부도여부.value_counts()

1.0    343
Name: 부도여부, dtype: int64

In [59]:
df.index

Index(['000360', '000830', '001300', '001310', '001370', '001830', '001950',
       '002530', '003040', '003600',
       ...
       '222520', '224090', '900010', '900020', '900030', '900060', '900150',
       '950010', '950030', '950070'],
      dtype='object', name='거래소코드', length=272)

In [60]:
pezi.index

Index(['001950', '002530', '004230', '004550', '008080', '010670', '011800',
       '012460', '014420', '014900',
       ...
       '077960', '036210', '007490', '014590', '057330', '008320', '045760',
       '006440', '058480', '011720'],
      dtype='object', name='거래소코드', length=343)

In [65]:
# 부도를 확정시킨 기업 데이터(pezi)를 활용하여 DART 공시와 비교할 기업의 데이터 수를 줄임
a = df[df.index.isin(pezi.index)==True]
a.index = a.index.astype('int')
a.index = a.index.map('{:06d}'.format)
a

,index
거래소코드,
001950,6
002530,7
004230,14
004550,15
008080,22
...,...
900060,267
900150,268
950010,269


In [71]:
a.to_csv('폐지사추가.xlsx')
# 이후 각 새로운 열을 만들어 각 폐지사마다 부도여부를 DART공시와 비교해가며 라벨링을 하고
# '폐지사1207'파일을 만듦